In [ ]:
import os
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import base64

# Get the absolute path of the current working directory
CURRENT_DIR = os.getcwd()
PASSWORD_FILE = os.path.join(CURRENT_DIR, "password_file.txt")

def hash_password(password):
    # Use PBKDF2 for password hashing
    salt = b'salt'  # Change this to a random value for production use
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        salt=salt,
        iterations=100000,
        length=32,
        backend=default_backend()
    )
    key = base64.urlsafe_b64encode(kdf.derive(password.encode()))
    return key.decode()

def read_password_file():
    try:
        with open(PASSWORD_FILE, 'r') as file:
            lines = file.readlines()
    except FileNotFoundError:
        # If the file doesn't exist, return an empty list
        return []
    return [line.strip().split(':') for line in lines]

def write_password_file(entries):
    with open(PASSWORD_FILE, 'w') as file:
        for entry in entries:
            file.write(':'.join(entry) + '\n')

def adduser():
    username = input("Enter username: ").strip().lower()
    real_name = input("Enter real name: ")
    password = input("Enter password: ")

    entries = read_password_file()

    # Check if the username already exists
    if any(entry[0] == username for entry in entries):
        print("Error: Username already exists.")
    else:
        hashed_password = hash_password(password)
        entries.append([username, real_name, hashed_password])
        write_password_file(entries)
        print(f"User {username} added successfully.")

def deluser():
    username = input("Enter username to delete: ").strip().lower()

    entries = read_password_file()

    # Remove the user if found
    filtered_entries = [entry for entry in entries if entry[0] != username]

    if len(filtered_entries) == len(entries):
        print("No user found with the given username.")
    else:
        write_password_file(filtered_entries)
        print(f"User {username} deleted successfully.")

def passwd():
    username = input("Enter username: ").strip().lower()
    current_password = input("Enter current password: ")
    new_password = input("Enter new password: ")
    confirm_password = input("Confirm new password: ")

    entries = read_password_file()

    # Find the user
    user_entry = next((entry for entry in entries if entry[0] == username), None)

    if user_entry is None or user_entry[2] != hash_password(current_password):
        print("Error: Invalid username or current password.")
    elif new_password != confirm_password:
        print("Error: New passwords do not match.")
    else:
        user_entry[2] = hash_password(new_password)
        write_password_file(entries)
        print(f"Password for user {username} changed successfully.")

def login():
    username = input("Enter username: ").strip().lower()
    password = input("Enter password: ")

    entries = read_password_file()

    # Find the user
    user_entry = next((entry for entry in entries if entry[0] == username), None)

    if user_entry is not None and user_entry[2] == hash_password(password):
        print("Login successful.")
    else:
        print("Login failed. Invalid username or password.")

# You can now call the functions as needed
adduser()

passwd()
login()
deluser()
